In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/alternate/sample_mapping.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


#### Make a dictionary of which k-mers belong to which genes

In [2]:
df_kir_ = pd.read_csv('/cellar/users/ramarty/Data/kir/components/tcga/kir_four_random_counts.agilent.good_coverage.csv',
                      index_col=0)

In [3]:
capture_kits = capture_kits[capture_kits.SHORT_ID.isin(df_kir_.index)]

In [5]:
# Median of the 100 random genes
random_genes = [x.strip() for x in open('/cellar/users/ramarty/Data/kir/kmers/reference_genes/random_genes.100.txt').readlines()]
len(random_genes)

9936

In [11]:
gene_names, sequences = [], []
for record in SeqIO.parse("/cellar/users/ramarty/Data/kir/ref/random_genes.fa", "fasta"):
    gene_names.append(record.id[:-1])
    sequences.append(str(record.seq))

In [12]:
random_df = pd.DataFrame({'Gene': gene_names,
                         'Sequence': sequences})

In [14]:
random_df.head()

,Gene,Sequence
0,ENST00000594070,AGTACACACAGGACCGTGGCTGCGGAATGCAGTTGAGGGCATCCCT...
1,ENST00000539759,CCAGTCAGGTGCTCCTGGGCTCCGGTCTCACCATCCTGTCCCAGCC...
2,ENST00000486489,CTGAATACAGGCTCAAAGCAGGCGACGGGACAGGAATGAGAGGATG...
3,ENST00000444179,GGCAGGCAGGCGCTGGGTCTGAGTACCCCTCTTTCCCCAGGGAGGC...
4,ENST00000503053,TGGCTGTAGCGACTGGTTGCGGGAGGCTGGACTCTGGTGACTGTGT...


In [46]:
random_gene_assignments = [[] for x in random_df.Gene]

for kmer in random_genes:
    for i, sequence in enumerate(random_df.Sequence):
        if kmer in sequence:
            random_gene_assignments[i].append(kmer)
            break

In [48]:
random_gene_assignments_dict = {}
for i, gene in enumerate(random_df.Gene):
    random_gene_assignments_dict[gene] = random_gene_assignments[i]

In [51]:
pickle.dump(random_gene_assignments_dict, 
            open('/cellar/users/ramarty/Data/kir/kmers/reference_genes/random_gene_assignments.p', 'w'))